# Todo
- Explore the bid-ask spread across different maturities and strikes
- Explore the bid-ask spread across different underlying assets
- Explore the spread wrt to vega, implied volatility, etc.
- Explore the spread for a single day
- See the impact of transaction cost on backtest result
  - Try implementing another tcost model
- Backtest delta hedging strategies
- Backtest delta-gamma hedging strategies
  - Try other variant on calls or puts.

In [ ]:
# Setup auto reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rc("font", **{"size": 18})
import numpy as np
from warnings import filterwarnings
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")

filterwarnings("ignore")

import investment_lab.option_selection as option_selection
import investment_lab.backtest as backtest
from investment_lab.data.option_db import OptionLoader
from investment_lab.data.rates_db import USRatesLoader
from investment_lab.option_trade import OptionTrade, DeltaHedgedOptionTrade, DeltaGammaHedgedOptionTrade
from investment_lab.backtest import StrategyBacktester, BacktesterBidAskFromData, BacktesterFixedRelativeBidAsk

from investment_lab.pricing.black_scholes import black_scholes_price, black_scholes_greeks
from investment_lab.rates import compute_forward
from investment_lab import option_strategies as option_strategies

In [ ]:
df_options = OptionLoader.load_data(datetime(2020, 1, 4), datetime(2022, 1, 4), process_kwargs={"ticker": "SPY"})
df_options.head()

In [ ]:
df_options["ask_bid_spread"] = df_options["ask"] - df_options["bid"]
df_options["ask_bid_spread_perc"] = (df_options["ask"] - df_options["bid"]) / df_options["mid"]

In [ ]:
df_options["day_to_expiration_bucket"] = pd.cut(
    df_options["day_to_expiration"],
    bins=[0, 7, 30, 90, 180, 360, 720, 1080],
    labels=["0-7", "8-30", "31-90", "91-180", "181-360", "361-720", "721-1080"],
)

df_options["delta_bucket"] = pd.cut(
    df_options["delta"],
    bins=[-1.0, -0.9, -0.75, -0.5, -0.25, 0, 0.1, 0.25, 0.5, 0.75, 0.9, 1],
    labels=[
        "[-1; -0.9]",
        "[-0.9; -0.75]",
        "[-0.75; -0.5]",
        "[-0.5; -0.25]",
        "[-0.25; 0]",
        "[0; 0.1]",
        "[0.1; 0.25]",
        "[0.25; 0.5]",
        "[0.5; 0.75]",
        "[0.75; 0.9]",
        "[0.9; 1]",
    ],
)


df_options["moneyness_bucket"] = pd.cut(
    df_options["moneyness"],
    bins=[0.5, 0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.25, 1.5],
    labels=[
        "[0.50; 0.75]",
        "[0.75; 0.85]",
        "[0.85; 0.90]",
        "[0.90; 0.95]",
        "[0.95; 1.00]",
        "[1.00; 1.05]",
        "[1.05; 1.10]",
        "[1.10; 1.25]",
        "[1.25; 1.50]",
    ],
)

In [ ]:
fig, (ax, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 8))
fig.suptitle("Bid-Ask Spread (in %) by Moneyness")
df_options[df_options["day_to_expiration_bucket"] == "0-7"].groupby(["moneyness_bucket", "call_put"])[["ask_bid_spread_perc"]].mean().unstack().plot.bar(
    xlabel="Moneyness Bucket", ylabel="Average Bid-Ask Spread in %", grid=True, title="Expiration 1-7days", ax=ax
)
ax.legend(fontsize=15)

df_options[df_options["day_to_expiration_bucket"] == "8-30"].groupby(["moneyness_bucket", "call_put"])[["ask_bid_spread_perc"]].mean().unstack().plot.bar(
    xlabel="Moneyness Bucket", ylabel="Average Bid-Ask Spread in %", grid=True, title="Expiration 8-30days", ax=ax2
)
ax2.legend(fontsize=15)


df_options[df_options["day_to_expiration_bucket"] == "181-360"].groupby(["moneyness_bucket", "call_put"])[
    ["ask_bid_spread_perc"]
].mean().unstack().plot.bar(xlabel="Moneyness Bucket", ylabel="Average Bid-Ask Spread in %", grid=True, title="Expiration 181-360days", ax=ax3)
ax3.legend(fontsize=15)

In [ ]:
fig, (ax, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 8))
fig.suptitle("Bid-Ask Spread (in %) by Delta")
df_options[df_options["day_to_expiration_bucket"] == "0-7"].groupby(["delta_bucket", "call_put"])[["ask_bid_spread_perc"]].mean().unstack().plot.bar(
    xlabel="Delta Bucket", ylabel="Average Bid-Ask Spread in %", grid=True, title="Expiration 1-7days", ax=ax
)
ax.legend(fontsize=15)

df_options[df_options["day_to_expiration_bucket"] == "8-30"].groupby(["delta_bucket", "call_put"])[["ask_bid_spread_perc"]].mean().unstack().plot.bar(
    xlabel="Delta Bucket", ylabel="Average Bid-Ask Spread in %", grid=True, title="Expiration 8-30days", ax=ax2
)
ax2.legend(fontsize=15)


df_options[df_options["day_to_expiration_bucket"] == "181-360"].groupby(["delta_bucket", "call_put"])[["ask_bid_spread_perc"]].mean().unstack().plot.bar(
    xlabel="Delta Bucket", ylabel="Average Bid-Ask Spread in %", grid=True, title="Expiration 181-360days", ax=ax3
)
ax3.legend(fontsize=15)

In [ ]:
fig, (ax, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 8))
fig.suptitle("Bid-Ask Spread (in $) by Moneyness")
df_options[df_options["day_to_expiration_bucket"] == "0-7"].groupby(["moneyness_bucket", "call_put"])[["ask_bid_spread"]].mean().unstack().plot.bar(
    xlabel="Moneyness Bucket", ylabel="Average Bid-Ask Spread in $", grid=True, title="Expiration 1-7days", ax=ax
)
ax.legend(fontsize=15)

df_options[df_options["day_to_expiration_bucket"] == "8-30"].groupby(["moneyness_bucket", "call_put"])[["ask_bid_spread"]].mean().unstack().plot.bar(
    xlabel="Moneyness Bucket", ylabel="Average Bid-Ask Spread in $", grid=True, title="Expiration 8-30days", ax=ax2
)
ax2.legend(fontsize=15)


df_options[df_options["day_to_expiration_bucket"] == "181-360"].groupby(["moneyness_bucket", "call_put"])[["ask_bid_spread"]].mean().unstack().plot.bar(
    xlabel="Moneyness Bucket", ylabel="Average Bid-Ask Spread in $", grid=True, title="Expiration 181-360days", ax=ax3
)
ax3.legend(fontsize=15)

In [ ]:
fig, (ax, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 8))
fig.suptitle("Bid-Ask Spread (in $) by Delta")
df_options[df_options["day_to_expiration_bucket"] == "0-7"].groupby(["delta_bucket", "call_put"])[["ask_bid_spread"]].mean().unstack().plot.bar(
    xlabel="Delta Bucket", ylabel="Average Bid-Ask Spread in $", grid=True, title="Expiration 1-7days", ax=ax
)
ax.legend(fontsize=15)

df_options[df_options["day_to_expiration_bucket"] == "8-30"].groupby(["delta_bucket", "call_put"])[["ask_bid_spread"]].mean().unstack().plot.bar(
    xlabel="Delta Bucket", ylabel="Average Bid-Ask Spread in $", grid=True, title="Expiration 8-30days", ax=ax2
)
ax2.legend(fontsize=15)


df_options[df_options["day_to_expiration_bucket"] == "181-360"].groupby(["delta_bucket", "call_put"])[["ask_bid_spread"]].mean().unstack().plot.bar(
    xlabel="Delta Bucket", ylabel="Average Bid-Ask Spread in $", grid=True, title="Expiration 181-360days", ax=ax3
)
ax3.legend(fontsize=15)

# Bid Ask Spread: P&L Impact


In [ ]:
df_SHORT_1W_STRANGLE_95_105 = OptionTrade.generate_trades(
    datetime(2020, 1, 2),
    datetime(2022, 12, 30),
    tickers="SPY",
    legs=option_strategies.SHORT_1W_STRANGLE_95_105,
    cost_neutral=False,
)

In [ ]:
backtest_SHORT_1W_STRANGLE_95_105 = StrategyBacktester(df_SHORT_1W_STRANGLE_95_105).compute_backtest()
backtest_SHORT_1W_STRANGLE_95_105_datatcost = BacktesterBidAskFromData(df_SHORT_1W_STRANGLE_95_105).compute_backtest()
backtest_SHORT_1W_STRANGLE_95_105_03tcost = BacktesterFixedRelativeBidAsk(df_SHORT_1W_STRANGLE_95_105).compute_backtest(
    tcost_args={"relative_half_spread": 0.03}
)

In [ ]:
from investment_lab.metrics.performance import sharpe_ratio, max_drawdown, calmar_ratio

In [ ]:
pd.DataFrame(
    [
        {
            "Name": name,
            "Maximum Drawdown": max_drawdown(nav["NAV"].pct_change().dropna()),
            "Sharpe Ratio": sharpe_ratio(nav["NAV"].pct_change().dropna()),
            "Calmar Ratio": calmar_ratio(nav["NAV"].pct_change().dropna()),
        }
        for name, nav in {
            "SHORT_1W_STRANGLE_95_105": backtest_SHORT_1W_STRANGLE_95_105.nav,
            "SHORT_1W_STRANGLE_95_105_data_tcost": backtest_SHORT_1W_STRANGLE_95_105_datatcost.nav,
            "SHORT_1W_STRANGLE_95_105_fixed_03tcost": backtest_SHORT_1W_STRANGLE_95_105_03tcost.nav,
        }.items()
    ]
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

backtest_SHORT_1W_STRANGLE_95_105.nav["NAV"].plot(ax=ax, label="No Transaction Cost")
backtest_SHORT_1W_STRANGLE_95_105_datatcost.nav["NAV"].plot(ax=ax, label="Data-based Transaction Cost")
backtest_SHORT_1W_STRANGLE_95_105_03tcost.nav["NAV"].plot(ax=ax, label="Fixed 5% Transaction Cost")
ax.legend()
ax.grid()

# Hedging


## Delta Hedging


In [ ]:
df_SHORT_1W_STRANGLE_95_105 = OptionTrade.generate_trades(
    datetime(2020, 1, 2),
    datetime(2021, 12, 30),
    tickers="SPY",
    legs=option_strategies.SHORT_1W_STRANGLE_95_105,
    cost_neutral=False,
)

## Delta-Gamma Hedging

Good hedging leg:

```python
"hedging_leg":{
    "day_to_expiry_target": 5,
    "strike_target": -0.1,
    "strike_col": "delta",
    "call_or_put": "P",
    "weight": 0.2,
    "leg_name": "Long 5D Call 5d",
    "rebal_week_day": [2],
}
```
